# Zero-Coupon Bond Using Marlowe Runtime\'s REST API (Liên kết bằng phiếu giảm giá bằng cách sử dụng API REST của Marlowe Runtime)

***Trước khi chạy sổ ghi chép này, bạn có thể muốn sử dụng chức năng "xóa đầu ra" của Jupyter để xóa kết quả thực hiện trước đó của sổ ghi chép này. Điều đó sẽ làm rõ hơn những gì đã được thực thi trong phiên hiện tại.***

Ví dụ về trái phiếu không INTEREST là một hợp đồng Marlowe đơn giản trong đó người cho vay cung cấp tiền gốc cho người đi vay, người này sẽ hoàn trả lại số tiền đó cùng với tiền lãi.

[Một video hoạt động thông qua notebook Jupyter này.](https://youtu.be/wgJVdkM2pBY)

Bạn có thể đặt câu hỏi về Marlowe trong [kênh #ask-marlowe trên IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) hoặc đăng các vấn đề với bài học này lên [sách các vấn đề của Bộ công cụ dành cho người mới bắt đầu Marlowe kho lưu trữ github.](https://github.com/input-output-hk/marlowe-starter-kit/issues).

Trong phần trình diễn này, chúng ta sử dụng giao diện dòng lệnh của Marlowe Runtime, `marlowe-runtime-cli`, để chạy hợp đồng này trên mạng thử nghiệm công khai `preprod` của Cardano. Hợp đồng Marlowe có thể sử dụng địa chỉ hoặc vai trò token để ủy quyền: ở đây chúng ta sử dụng địa chỉ.

Trong [Marlowe Playground](https://play.marlowe-finance.io/), hợp đồng có dạng như sau ở định dạng Blockly.

![Zero-coupon bond Marlowe contract](images/02-zcb-contract.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Role "Lender")
            (Role "Lender")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Role "Lender")
            (Party (Role "Borrower"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Role "Borrower")
                        (Role "Borrower")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Role "Borrower")
                        (Party (Role "Lender"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close
```

## Chuẩn bị

Xem [Lesson 0. Preliminaries](00-preliminaries.md) để biết thông tin về cách thiết lập môi trường của một người để sử dụng hướng dẫn này.

Bài học giả định rằng các biến môi trường sau đây đã được thiết lập.

- `CARDANO_NODE_SOCKET_PATH`: vị trí socket của node Cardano.
- `CARDANO_TESTNET_MAGIC`: số magic testnet.
- `MARLOWE_RT_WEBSERVER_HOST`: Địa chỉ IP của máy chủ proxy Marlowe Runtime.
- `MARLOWE_RT_WEBSERVER_PORT`: Số cổng cho máy chủ proxy Marlowe Runtime.

Nó cũng giả định rằng các bên Bên cho vay và Bên vay có địa chỉ, khóa ký và tiền.
- Người cho vay
    - [keys/lender.address](keys/lender.address): Địa chỉ Cardano của người cho vay
    - [keys/lender.skey](keys/lender.skey): vị trí ký tệp khóa cho bên cho vay
- Người vay
    - [keys/borrower.address](keys/borrower.address): Địa chỉ Cardano cho người vay
    - [keys/borrower.skey](keys/borrower.skey): vị trí ký file key cho bên vay

### Truy cập vào node Cardano và Runtimes Marlowe

Nếu chúng ta đang sử dụng tiện ích mở rộng Cardano Marlowe Runtime của [demeter.run](https://demeter.run/) thì chúng ta đã có quyền truy cập vào Cardano Node và Marlowe Runtime. Các lệnh sau sẽ đặt các biến môi trường cần thiết để sử dụng triển khai docker cục bộ trên các cổng mặc định. Nó cũng sẽ thiết lập một số biến môi trường bổ sung.

In [1]:
if [[ -z "$MARLOWE_RT_WEBSERVER_PORT" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for Marlowe Runtime REST API.
  export MARLOWE_RT_WEBSERVER_HOST="127.0.0.1"
  export MARLOWE_RT_WEBSERVER_PORT=3780

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

MARLOWE_RT_WEBSERVER_URL="http://$MARLOWE_RT_WEBSERVER_HOST":"$MARLOWE_RT_WEBSERVER_PORT"

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_WEBSERVER_HOST = $MARLOWE_RT_WEBSERVER_HOST"
echo "MARLOWE_RT_WEBSERVER_PORT = $MARLOWE_RT_WEBSERVER_PORT"
echo "MARLOWE_RT_WEBSERVER_URL = $MARLOWE_RT_WEBSERVER_URL"

CARDANO_NODE_SOCKET_PATH = ~/.local/share/containers/storage/volumes/marlowe-starter-kit_shared/_data/node.socket
CARDANO_TESTNET_MAGIC = 1
MARLOWE_RT_WEBSERVER_HOST = 127.0.0.1
MARLOWE_RT_WEBSERVER_PORT = 3780
MARLOWE_RT_WEBSERVER_URL = http://127.0.0.1:3780


Lưu ý số magic mạng thử nghiệm:
- `preprod` = 1
- `preview` = 2

### Địa chỉ và quỹ của bên cho vay

Kiểm tra xem địa chỉ và khóa đã được tạo cho người cho vay chưa. Nếu không, hãy xem phần "Tạo địa chỉ và khóa ký" trong [Lesson 0. Preliminaries](00-preliminaries.md).

In [2]:
LENDER_SKEY=keys/lender.skey
LENDER_ADDR=$(cat keys/lender.address)
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


Kiểm tra xem người cho vay có ít nhất một trăm ADA không.

In [3]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b8be3cb7f1e387578d37da01adaa3186e9814e6557c1ee9915ac5039eb4277fb     1        1000000000 lovelace + TxOutDatumNone


Người ta có thể xem địa chỉ trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [4]:
echo "$EXPLORER_URL"/address/"$LENDER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### Địa chỉ và tiền của người vay

Kiểm tra xem địa chỉ và khóa đã được tạo cho người mượn chưa. Nếu không, hãy xem phần "Tạo địa chỉ và khóa ký" trong [Lesson 0. Preliminaries](00-preliminaries.md).

In [5]:
BORROWER_SKEY=keys/borrower.skey
BORROWER_ADDR=$(cat keys/borrower.address)
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


Kiểm tra xem người vay có ít nhất một trăm ADA không.

In [6]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b8be3cb7f1e387578d37da01adaa3186e9814e6557c1ee9915ac5039eb4277fb     2        1000000000 lovelace + TxOutDatumNone


Người ta có thể xem địa chỉ trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [7]:
echo "$EXPLORER_URL"/address/"$BORROWER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


## Thiết kế hợp đồng

Bạn có thể tải xuống hợp đồng trái phiếu không nhận phiếu thưởng từ [Marlowe Playground](https://play.marlowe-finance.io/) dưới dạng tệp JSON hoặc có thể tạo bằng cách sử dụng [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) bằng cách sử dụng lệnh `marlowe-cli template`.

Đặt tiền gốc của khoản vay là 80 ADA và INTEREST là 5 ADA.

In [8]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))
echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

PRINCIPAL = 80000000 lovelace
INTEREST = 5000000 lovelace


Trên blockchain Cardano, các tham số giao thức yêu cầu mỗi UTxO chứa ít nhất một số ADA. Ở đây chúng ta sẽ bắt đầu hợp đồng với 2 ADA.

In [9]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Ở phần sau của ví dụ, chúng ta sẽ cần một số hằng số để chuyển đổi thời gian.

In [10]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

### *Phương án 1:* Sử dụng Marlowe Playground để thiết kế hợp đồng

Nếu bạn muốn tạo hợp đồng trong Marlowe Playground, hãy làm như sau:

1. Truy cập https://play.marlowe-finance.io/ trong trình duyệt web.
2. Chọn  "Open an Example".
3. Chọn  "Marlowe" hoặc "Blockly" trong "Zero Coupon Bond".
4. Chọn  "Send to Simulator".
5. Đặt "Loan Deadline" thành một giờ trong tương lai.
6. Đặt "Payback Deadline" thành ba giờ trong tương lai.
7. Đặt "Principal" thành 80 ADA.
8. Đặt "Interest"  thành 5 ADA.
9. Chọn  "Download as JSON", đặt tên tệp thành "zcb-contract.json" và lưu trữ tệp trong thư mục này, cụ thể là [marlowe-starter-kit/01-runtime-rest/](.).

![Setting parameters for the zero-coupon bond contract in Marlowe Playground](images/zcb-playground.png)

### *Phương án 2:* Sử dụng Marlowe CLI để tạo hợp đồng

Dưới đây chúng ta tạo hợp đồng bằng Marlowe CLI.

Trước tiên, hãy tìm thời gian hiện tại, được đo bằng [POSIX milliseconds](https://en.wikipedia.org/wiki/Unix_time).

In [11]:
NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"

NOW = 1679604689000 POSIX milliseconds = Thu Mar 23 02:51:29 PM MDT 2023


Hợp đồng có thời hạn cho vay và thời hạn trả nợ. Để thuận tiện trong ví dụ này, hãy đặt thời hạn cho tương lai gần.

In [12]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo LENDER_DEADLINE = "$LENDER_DEADLINE" POSIX milliseconds = "`date -d @$((LENDER_DEADLINE / SECOND))`"
echo BORROWER_DEADLINE = "$BORROWER_DEADLINE" POSIX milliseconds = "`date -d @$((BORROWER_DEADLINE / SECOND))`"

LENDER_DEADLINE = 1679608289000 POSIX milliseconds = Thu Mar 23 03:51:29 PM MDT 2023
BORROWER_DEADLINE = 1679615489000 POSIX milliseconds = Thu Mar 23 05:51:29 PM MDT 2023


Bây giờ hãy tạo tệp JSON cho hợp đồng, `zcb-contract.json`.

In [13]:
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender Lender \
  --borrower Borrower \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file /dev/null

Các tùy chọn dòng lệnh khác nhau được mô tả bởi hệ thống trợ giúp.

In [14]:
marlowe-cli template zcb --help

Usage: marlowe-cli template zcb --minimum-ada INTEGER --lender PARTY
                                --borrower PARTY --principal INTEGER
                                --interest INTEGER --lending-deadline TIMEOUT
                                --repayment-deadline TIMEOUT

  Create a zero-coupon bond.

Available options:
  --minimum-ada INTEGER    Lovelace that the lender contributes to the initial
                           state.
  --lender PARTY           The lender.
  --borrower PARTY         The borrower.
  --principal INTEGER      The principal, in lovelace.
  --interest INTEGER       The interest, in lovelace.
  --lending-deadline TIMEOUT
                           The lending deadline. POSIX milliseconds or duration:
                           `INTEGER[s|m|d|w|h]`.
  --repayment-deadline TIMEOUT
                           The repayment deadline. POSIX milliseconds or
                           duration: `INTEGER[s|m|d|w|h]`.
  -h,--help                Show this help text


## Kiểm tra hợp đồng

Xem tệp hợp đồng dưới dạng YAML.

In [15]:
json2yaml zcb-contract.json

timeout: 1679608289000
timeout_continuation: close
when:
- case:
    deposits: 80000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Lender
  then:
    from_account:
      role_token: Lender
    pay: 80000000
    then:
      timeout: 1679615489000
      timeout_continuation: close
      when:
      - case:
          deposits:
            add: 80000000
            and: 5000000
          into_account:
            role_token: Borrower
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            role_token: Borrower
        then:
          from_account:
            role_token: Borrower
          pay:
            add: 80000000
            and: 5000000
          then: close
          to:
            party:
              role_token: Lender
          token:
            currency_symbol: ''
            token_name: ''
    to:
      party:
        role_token: Borro

### \[Không bắt buộc, nhưng nên dùng\] Kiểm tra độ an toàn của hợp đồng

Nếu chúng ta đang chạy hợp đồng trên Cardano `mainnet`, thì chúng ta muốn kiểm tra tính an toàn của nó trước khi tạo nó, để không có khả năng chúng ta có thể mất tiền.

Dưới đây là các bước để kiểm tra mức độ an toàn của hợp đồng:

1. Hiểu [Ngôn ngữ Marlowe](https://marlowe-finance.io/).
2. Hiểu [Mô hình UTxO mở rộng](https://docs.cardano.org/learn/eutxo-explainer) của Cardano.
3. Đọc và hiểu [Hướng dẫn về các phương pháp hay nhất của Marlowe](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Đọc và hiểu [Hướng dẫn bảo mật của Marlowe](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Sử dụng [Marlowe Playground](https://play.marlowe-finance.io/) để gắn cờ cảnh báo, thực hiện phân tích tĩnh và mô phỏng hợp đồng.
6. Sử dụng [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` công cụ để nghiên cứu xem hợp đồng có thể chạy trên mạng Cardano hay không.
7. Chạy *tất cả đường dẫn thực thi* của hợp đồng trên [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

Xem [Bài 1](01-runtime-cli.ipynb) để biết ví dụ về cách thực hiện bước 6.

## Giao dịch 1. Tạo Hợp đồng

Yêu cầu `HTTP` `POST` đối với điểm cuối `/contracts` của Marlowe Runtime\ sẽ xây dựng giao dịch tạo cho hợp đồng Marlowe. Chúng ta cung cấp cho nó tệp JSON chứa hợp đồng và cho nó biết giá trị `MIN_LOVELACE` mà chúng ta đã chọn trước đó. Bất kỳ ai cũng có thể tạo hợp đồng, nhưng trong ví dụ này, người cho vay sẽ làm như vậy, vì vậy chúng ta cung cấp địa chỉ của họ để tài trợ cho giao dịch và nhận tiền thay đổi từ đó.

Trước tiên, chúng ta tạo phần thân JSON của yêu cầu để xây dựng giao dịch tạo.

In [16]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat zcb-contract.json`
roles:
  Lender: "$LENDER_ADDR"
  Borrower: "$BORROWER_ADDR"
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json

{"contract":{"timeout":1679608289000,"timeout_continuation":"close","when":[{"case":{"deposits":80000000,"into_account":{"role_token":"Lender"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Lender"}},"then":{"from_account":{"role_token":"Lender"},"pay":80000000,"then":{"timeout":1679615489000,"timeout_continuation":"close","when":[{"case":{"deposits":{"add":80000000,"and":5000000},"into_account":{"role_token":"Borrower"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Borrower"}},"then":{"from_account":{"role_token":"Borrower"},"pay":{"add":80000000,"and":5000000},"then":"close","to":{"party":{"role_token":"Lender"}},"token":{"currency_symbol":"","token_name":""}}}]},"to":{"party":{"role_token":"Borrower"}},"token":{"currency_symbol":"","token_name":""}}}]},"metadata":{},"minUTxODeposit":2000000,"roles":{"Borrower":"addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d","Lender":"addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2

Tiếp theo, chúng ta đăng yêu cầu và xem phản hồi.

In [17]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

links:
  contract: contracts/bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d%231
resource:
  contractId: bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1
  txBody:
    cborHex: 86a80081825820b8be3cb7f1e387578d37da01adaa3186e9814e6557c1ee9915ac5039eb4277fb010d81825820b8be3cb7f1e387578d37da01adaa3186e9814e6557c1ee9915ac5039eb4277fb010184a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b55c4daa300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820cc80fcdc9dedea89d2f50dd29cf7f84d14bd00e27714310a0cfe5e24e87fd008a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a000fea4ca1581c89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43ca148426f72726f77657201a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a000fc8a0a1581c89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43ca1464c656e6465720110a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b

Mã định danh cho hợp đồng được nhúng trong phản hồi.

In [18]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1


Việc xê-ri hóa CBOR (ở định dạng phong bì văn bản) cũng được nhúng trong phản hồi.

In [19]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

Có nhiều cách để ký và gửi giao dịch Cardano:
- `cardano-cli` tại dòng lệnh
- `cardano-wallet` tại dòng lệnh hoặc dưới dạng dịch vụ REST
- `cardano-hw-cli` đối với ví phần cứng tại dòng lệnh
- ví CIP-30 tương thích với Babbage trong trình duyệt web
- `marlowe-cli` tại dòng lệnh

Để thuận tiện, ở đây chúng ta sử dụng `marlowe-cli transaction submit`. Người ta có thể phải đợi một phút hoặc lâu hơn để các giao dịch được xác nhận trên blockchain.

In [20]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

TX_1 = bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d


Người ta có thể xem giao dịch trên Cardano explorer và thấy rằng hợp đồng đã được tạo và các bên đã nhận được vai trò token của họ. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [21]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d?tab=utxo


Cụ thể, chúng ta thấy rằng hợp đồng Marlowe giữ 2 ADA đã được đặt là `MINIMUM_LOVELACE`.

In [22]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "cc80fcdc9dedea89d2f50dd29cf7f84d14bd00e27714310a0cfe5e24e87fd008"


Người ta có thể thấy rằng người cho vay và người vay đã nhận được vai trò token của họ. Lưu ý rằng `4c656e646572 = Lender` và `426f72726f776572 = Borrower` ở dạng ký hiệu thập lục phân.

In [23]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d     0        995476698 lovelace + TxOutDatumNone
bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d     3        1034400 lovelace + 1 89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43c.4c656e646572 + TxOutDatumNone


In [24]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b8be3cb7f1e387578d37da01adaa3186e9814e6557c1ee9915ac5039eb4277fb     2        1000000000 lovelace + TxOutDatumNone
bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d     2        1043020 lovelace + 1 89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43c.426f72726f776572 + TxOutDatumNone


## Xem chi tiết của hợp đồng trên blockchain

Marlowe Runtime\'s `HTTP` `GET` với endpoint `/contracts/{contractId}` có thể truy vấn và trả về thông tin về hợp đồng đó.

In [25]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://127.0.0.1:3780/contracts/bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d%231


In [26]:
curl -sS "$CONTRACT_URL" | json2yaml

links:
  transactions: contracts/bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d%231/transactions
resource:
  block:
    blockHeaderHash: 4f1c23e34773b2780bb4507770740fb9ef569d38e08e8c6006964f3d71677529
    blockNo: 755129
    slotNo: 23921744
  continuations: null
  contractId: bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1
  currentContract:
    timeout: 1679608289000
    timeout_continuation: close
    when:
    - case:
        deposits: 80000000
        into_account:
          role_token: Lender
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Lender
      then:
        from_account:
          role_token: Lender
        pay: 80000000
        then:
          timeout: 1679615489000
          timeout_continuation: close
          when:
          - case:
              deposits:
                add: 80000000
                and: 5000000
              into_account:
                role_token

## Giao dịch 2. Người cho vay gửi vốn của chính mình

Người cho vay gửi 80 ada của mình vào hợp đồng bằng cách sử dụng endpoint `HTTP` `POST` `/contract/{contractId}/transactions` của Marlowe Runtime. Người cho vay cung cấp nguồn vốn và nhận được tiền thừa từ giao dịch này, vì vậy chúng tôi cung cấp địa chỉ của họ.

Số tiền gửi được biểu diễn dưới dạng đầu vào JSON cho hợp đồng. Công cụ `marlowe-cli input deposit` sẽ tự động định dạng JSON đúng cho một khoản gửi.

In [27]:
marlowe-cli input deposit --help

Usage: marlowe-cli input deposit --deposit-account PARTY --deposit-party PARTY 
                                 [--deposit-token TOKEN]
                                 --deposit-amount INTEGER 
                                 [--out-file OUTPUT_FILE]

  Create Marlowe input for a deposit.

Available options:
  --deposit-account PARTY  The account for the deposit.
  --deposit-party PARTY    The party making the deposit.
  --deposit-token TOKEN    The token being deposited, if not Ada.
  --deposit-amount INTEGER The amount of token being deposited.
  --out-file OUTPUT_FILE   JSON output file for contract input.
  -h,--help                Show this help text


In [28]:
marlowe-cli input deposit \
  --deposit-party Lender \
  --deposit-account Lender \
  --deposit-amount "$PRINCIPAL" \
  --out-file input-2.json
json2yaml input-2.json

input_from_party:
  role_token: Lender
into_account:
  role_token: Lender
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 80000000


Đầu vào này được bao gồm trong yêu cầu JSON.

In [29]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

{"inputs":[{"input_from_party":{"role_token":"Lender"},"into_account":{"role_token":"Lender"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":80000000}],"metadata":{},"tags":{},"version":"v1"}


Tiếp theo, chúng ta đăng yêu cầu và lưu trữ phản hồi.

In [30]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

links:
  transaction: contracts/bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d%231/transactions/088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14
resource:
  contractId: bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1
  transactionId: 088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14
  txBody:
    cborHex: 86aa0083825820bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d00825820bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d01825820bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d030d81825820bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010184a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a36844ba7a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e848002820058206db2f6b791dcab9fca1b17ccfe35d0a486c260fc300c48d14e24f78e90da6b86a200581d601b120

Một lần nữa, hãy sử dụng `marlowe-cli` để gửi giao dịch rồi chờ xác nhận.

In [31]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

In [32]:
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = 088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14


Người ta có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [33]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14?tab=utxo


Người ta có thể thấy rằng người cho vay có ít hơn khoảng 83 ADA so với ban đầu. Hai ADA đã được gửi vào hợp đồng khi nó được tạo ra và 80 ADA đã được trả cho người vay trong giao dịch thứ hai; 1 ADA khác được gắn vào vai trò token đã được gửi cho người vay. Người cho vay cũng giữ vai trò token của riêng họ.

In [34]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14     0        914639783 lovelace + TxOutDatumNone
088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14     2        1034400 lovelace + 1 89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43c.4c656e646572 + TxOutDatumNone


Hợp đồng Marlowe vẫn có 2 ADA từ khi tạo ra nó.

In [35]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "6db2f6b791dcab9fca1b17ccfe35d0a486c260fc300c48d14e24f78e90da6b86"


Địa chỉ thanh toán theo vai trò của Marlowe giữ 80 ADA thay mặt cho người đi vay.

In [36]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#3"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14     3        80000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "12548b4001cc30918dc86060c05d1cafe4457ec1e9f0e271d022cd1edddd814f"


## Xem thêm tiến trình của hợp đồng trên blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` có thể tìm nạp hợp đồng từ blockchain và trả lại thông tin về hợp đồng đó.

In [37]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_2" | json2yaml

links: {}
resource:
  block:
    blockHeaderHash: 63ffbfd04ef1c76a2209ec29d9f512e39717aecb6715ae89c9901513a9317d2b
    blockNo: 755144
    slotNo: 23922080
  consumingTx: null
  continuations: null
  contractId: bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1
  inputUtxo: bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1
  inputs:
  - input_from_party:
      role_token: Lender
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 80000000
  invalidBefore: 2023-03-23T21:00:14Z
  invalidHereafter: 2023-03-23T21:51:29Z
  metadata: {}
  outputContract:
    timeout: 1679615489000
    timeout_continuation: close
    when:
    - case:
        deposits:
          add: 80000000
          and: 5000000
        into_account:
          role_token: Borrower
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Borrower
      then:
        fr

## Giao dịch 3. Người vay rút tiền gốc

Tiền gốc của 80 ADA được giữ tại địa chỉ thanh toán theo vai trò của Marlowe vì lợi ích của người đi vay. Người vay có thể rút các khoản tiền này bất cứ lúc nào. ID hợp đồng và tên vai trò được bao gồm trong nội dung yêu cầu rút tiền.

In [38]:
yaml2json << EOI > request-3.json
contractId: "$CONTRACT_ID"
role: Borrower
EOI
cat request-3.json

{"contractId":"bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1","role":"Borrower"}


Tiếp theo, chúng ta đăng yêu cầu và lưu trữ phản hồi.

In [39]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BORROWER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

links:
  withdrawal: withdrawals/186e94a9721b90a980171ac8b6c69553bfd18fb72ef7eab5184f921b01ea90ad
resource:
  txBody:
    cborHex: 86a80083825820088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c1403825820b8be3cb7f1e387578d37da01adaa3186e9814e6557c1ee9915ac5039eb4277fb02825820bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d020d81825820b8be3cb7f1e387578d37da01adaa3186e9814e6557c1ee9915ac5039eb4277fb0212818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020182a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a405a5e50a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a000fea4ca1581c89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43ca148426f72726f7765720110a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3b931a78111a0007af88021a00051fb00b58208558da947714b89978d96dc272163b76a766b6c9d1ee06e39cdb5c41f7247d589fff81d8799f581c89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43c4

Một lần nữa, hãy sử dụng `marlowe-cli` để gửi giao dịch rồi chờ xác nhận.

In [40]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

In [41]:
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = 186e94a9721b90a980171ac8b6c69553bfd18fb72ef7eab5184f921b01ea90ad


Bật có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [42]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/186e94a9721b90a980171ac8b6c69553bfd18fb72ef7eab5184f921b01ea90ad?tab=utxo


Người vay hiện có thêm khoảng 80 ADA (tiền gốc của khoản vay). Người vay cũng giữ vai trò token của riêng họ.

In [43]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
186e94a9721b90a980171ac8b6c69553bfd18fb72ef7eab5184f921b01ea90ad     0        1079664208 lovelace + TxOutDatumNone
186e94a9721b90a980171ac8b6c69553bfd18fb72ef7eab5184f921b01ea90ad     1        1043020 lovelace + 1 89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43c.426f72726f776572 + TxOutDatumNone


## Giao dịch 4. Người vay hoàn trả khoản vay

Sau một thời gian, người vay trả lại tiền gốc cộng với tiền lãi. Vì vậy, họ tài trợ cho giao dịch và nhận tiền thay đổi tại địa chỉ của họ.

Đầu tiên xây dựng đầu vào để gửi tiền vào hợp đồng.

In [44]:
marlowe-cli input deposit \
  --deposit-party Borrower \
  --deposit-account Borrower \
  --deposit-amount "$((PRINCIPAL+INTEREST))" \
  --out-file input-4.json
json2yaml input-4.json

input_from_party:
  role_token: Borrower
into_account:
  role_token: Borrower
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 85000000


Tiếp theo xây dựng yêu cầu.

In [45]:
yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat input-4.json)]
metadata: {}
tags: {}
EOI
cat request-4.json

{"inputs":[{"input_from_party":{"role_token":"Borrower"},"into_account":{"role_token":"Borrower"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":85000000}],"metadata":{},"tags":{},"version":"v1"}


Bây giờ hãy đăng yêu cầu lên Marlowe Runtime.

In [46]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BORROWER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

links:
  transaction: contracts/bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d%231/transactions/d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047
resource:
  contractId: bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1
  transactionId: d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047
  txBody:
    cborHex: 86aa0083825820088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c1401825820186e94a9721b90a980171ac8b6c69553bfd18fb72ef7eab5184f921b01ea90ad00825820186e94a9721b90a980171ac8b6c69553bfd18fb72ef7eab5184f921b01ea90ad010d81825820186e94a9721b90a980171ac8b6c69553bfd18fb72ef7eab5184f921b01ea90ad0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010184a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3b3f4709a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a000fea4ca1581c89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43ca148426f72726f77657201a30

Một lần nữa, hãy sử dụng `marlowe-cli` để gửi giao dịch rồi chờ xác nhận.

In [47]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

In [48]:
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047


Người ta có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [49]:
echo "$EXPLORER_URL"/transaction/"$TX_4?tab=utxo"

https://preprod.cardanoscan.io/transaction/d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047?tab=utxo


Người vay hiện có tổng cộng khoảng 5 ADA (tiền lãi của khoản vay) ít hơn so với ban đầu.

In [50]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047     0        994002697 lovelace + TxOutDatumNone
d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047     1        1043020 lovelace + 1 89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43c.426f72726f776572 + TxOutDatumNone


85 ADA ở địa chỉ thanh toán theo vai trò của Marlowe\, được giữ vì lợi ích của người cho vay.

In [51]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_4#2"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047     2        85000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "e339bd7d8c4dad56433ece9812d8ca16fd29544aa2246450f53a9b7bfb73775f"


Hợp đồng Marlowe đã đóng, vì vậy không có đầu ra nào cho địa chỉ tập lệnh của nó.

## Xem việc hoàn thành hợp đồng trên blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` có thể tìm nạp hợp đồng từ blockchain và trả lại thông tin về hợp đồng đó.

In [52]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_4" | json2yaml

links:
  previous: contracts/bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d%231/transactions/088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14
resource:
  block:
    blockHeaderHash: eb0c3b71748c48f4e11d57f79999f32b76389b3d393626f7a25cd182942e9952
    blockNo: 755163
    slotNo: 23922472
  consumingTx: null
  continuations: null
  contractId: bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1
  inputUtxo: 088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c14#1
  inputs:
  - input_from_party:
      role_token: Borrower
    into_account:
      role_token: Borrower
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 85000000
  invalidBefore: 2023-03-23T21:07:14Z
  invalidHereafter: 2023-03-23T23:51:29Z
  metadata: {}
  outputContract: null
  outputState: null
  outputUtxo: null
  status: confirmed
  tags: {}
  transactionId: d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047
  txBody: nul

## Giao dịch 5. Người cho vay rút tiền gốc và tiền lãi

Ở đây một lần nữa, chúng ta rút tiền từ địa chỉ xuất chi theo vai trò của Marlowe.

In [53]:
yaml2json << EOI > request-5.json
contractId: "$CONTRACT_ID"
role: Lender
EOI
cat request-5.json

{"contractId":"bfb6db4bdf59113c4bdd45d45c7f94070099b813e83b665b5cdc6daeb91a941d#1","role":"Lender"}


Tiếp theo, chúng ta đăng yêu cầu và lưu trữ phản hồi.

In [54]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

links:
  withdrawal: withdrawals/32487203bba96c85e34345e8f88fc60dfb12dac4fb89decf144b86287a1a65e6
resource:
  txBody:
    cborHex: 86a80083825820088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c1400825820088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c1402825820d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047020d81825820088395d43bd1aefc7a5b1dc41b472a74f59cca631a61af7c9a212c5c8fa56c140012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020182a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b903bc5a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a000fc8a0a1581c89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43ca1464c656e6465720110a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a367cb4f4111a000796b3021a00050f220b5820499b9851886e239567b2594375fe9ec5857dd60c60bd3251b2ebe39cf6e834bd9fff81d8799f581c89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43c464c6

Một lần nữa, hãy sử dụng `marlowe-cli` để gửi giao dịch rồi chờ xác nhận.

In [55]:
jq '.resource.txBody' response-5.json > tx-5.unsigned

In [56]:
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

TX_5 = 32487203bba96c85e34345e8f88fc60dfb12dac4fb89decf144b86287a1a65e6


Người ta có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [57]:
echo "$EXPLORER_URL"/transaction/"$TX_5?tab=utxo"

https://preprod.cardanoscan.io/transaction/32487203bba96c85e34345e8f88fc60dfb12dac4fb89decf144b86287a1a65e6?tab=utxo


Người cho vay hiện có thêm 5 ADA (lãi của khoản vay), trừ phí, so với số dư ban đầu của họ trước khi hợp đồng bắt đầu.

In [58]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
32487203bba96c85e34345e8f88fc60dfb12dac4fb89decf144b86287a1a65e6     0        999308229 lovelace + TxOutDatumNone
32487203bba96c85e34345e8f88fc60dfb12dac4fb89decf144b86287a1a65e6     1        1034400 lovelace + 1 89506f74616aa4b5a42ec352a56ab7815e547e23e2a32417f400e43c.4c656e646572 + TxOutDatumNone
d08277e718607803947c16d13d6ba78a464d48667cb08f199fbf3257791f7047     3        2000000 lovelace + TxOutDatumNone


## *Phụ lục:* Mô tả OpenAPI của Dịch vụ web Marlowe

Dưới đây là cách lấy mô tả OpenAPI của các điểm cuối và loại do `marlowe-web-server` cung cấp..

In [59]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL/openapi.json" | json2yaml | head -n 20

components:
  schemas:
    Action:
      description: A contract which becomes active when an action occurs.
      oneOf:
      - properties:
          deposits:
            $ref: '#/components/schemas/Value'
          into_account:
            $ref: '#/components/schemas/Party'
          of_token:
            $ref: '#/components/schemas/Token'
          party:
            $ref: '#/components/schemas/Party'
        required:
        - party
        - deposits
        - of_token
        - into_account
        type: object


Nó cũng có sẵn trên trang https://marlowe-finance.io/docs/development/runtime-rest-api/.

![Marlowe Runtime OpenAPI](images/openapi.png)